# Initialization

## modules

In [1]:
from case_study.integrations.database import  connections
from case_study.integrations.duckdb import DuckDBIntegrator
from case_study.ai_join_advisor import SmartJoinBuilder
from case_study.jupyter_ui import select_databases_widget,select_tables_widget

2025-04-23 19:13:22,752 - database-queries - INFO - Initializing ConnectionConfig with path: config.yaml
2025-04-23 19:13:23,018 - database-queries - INFO - Loading configuration from config.yaml
2025-04-23 19:13:23,240 - database-queries - INFO - Processing connection configuration for: postgres_datamancers
2025-04-23 19:13:23,375 - database-queries - INFO - Successfully configured connection: postgres_datamancers
2025-04-23 19:13:23,582 - database-queries - INFO - Processing connection configuration for: datamancers
2025-04-23 19:13:23,866 - database-queries - INFO - Successfully configured connection: datamancers
2025-04-23 19:13:24,037 - database-queries - INFO - ConnectionConfig initialization completed


## Required objects
- SmartJoinBuilder() - class that suggest join between two selected tables
- DuckDbIntegrator() - class for querying across multiple (types) of databases within single query  

In [2]:
builder = SmartJoinBuilder()
query_engine = DuckDBIntegrator()

2025-04-23 19:13:26,460 - join-advisor - INFO - Initializing SmartJoinBuilder with config path: config.yaml
2025-04-23 19:13:26,904 - duckdb-integrator - INFO - Initializing DuckDB integrator with config path: config.yaml
2025-04-23 19:13:27,029 - duckdb-integrator - INFO - Using DuckDB database path: data/duckdb_logs.db
2025-04-23 19:13:27,184 - duckdb-integrator - INFO - Ensured database directory exists: data
2025-04-23 19:13:27,448 - duckdb-integrator - INFO - DuckDB connection established
2025-04-23 19:13:27,548 - database-queries - INFO - Initializing ConnectionConfig with path: config.yaml
2025-04-23 19:13:27,744 - database-queries - INFO - Loading configuration from config.yaml
2025-04-23 19:13:27,877 - database-queries - INFO - Processing connection configuration for: postgres_datamancers
2025-04-23 19:13:28,030 - database-queries - INFO - Successfully configured connection: postgres_datamancers
2025-04-23 19:13:28,245 - database-queries - INFO - Processing connection configur

# Workflow
1. select databases to connect from your configuration file 
2. Smart Join Suggest (optional): do you want to join two tables across your connected databases but don't know how? Select them and let the ai give you the suggestion!
3. Querying engine: Run your query across databases! You can query any data across your connections, just wrap the table reference into "DB type function" (e.g. BIGQUERY(project-id.dataset_id.table_id)) and let the magic flow! You can also join tables across databases!

In [ ]:
select_databases_widget(connections,builder)

In [ ]:
select_tables_widget(builder)

In [ ]:
query="""WITH
    bq_data AS (
        SELECT *, ts_end-ts_start AS duration FROM BIGQUERY('datamancers.dbt.L1_POSTGRES__JOBS__planned_cleaned')
    ), pg_data_pricings AS (
        SELECT * FROM POSTGRES('public.project_resources_pricings')),
       pg_data_pricings AS (
        SELECT * FROM POSTGRES('public.project_resources_pricings')
    
        SELECT * FROM bq_data 
        INNER JOIN pg_data 
        ON pg_data.project_id = bq_data.project_id
        AND pg_data.client_id = bq_data.client_id
        AND pg_data.resource_id = bq_data.resource_id
        AND bq_data.ts_start BETWEEN pg_data.date_from AND COALESCE(pg_data.date_to, CURRENT_DATE)

        """

data=query_engine.run_query(query)
data

/Users/vitmrnavek/projects/datamancers/clients/rohlik/case_study/venv/lib/python3.11/site-packages/google/cloud/bigquery/table.py:1933: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


2025-04-23 19:22:35,123 - duckdb-integrator - INFO - Logging query performance for source: bigquery
2025-04-23 19:22:35,330 - duckdb-integrator - INFO - Query performance logged successfully
2025-04-23 19:22:36,094 - duckdb-integrator - INFO - Logging query performance for source: postgres
2025-04-23 19:22:36,326 - duckdb-integrator - INFO - Query performance logged successfully
2025-04-23 19:22:36,959 - duckdb-integrator - INFO - Logging query performance for source: duckdb
2025-04-23 19:22:37,162 - duckdb-integrator - INFO - Query performance logged successfully
2025-04-23 19:22:37,378 - duckdb-integrator - INFO - Starting duplicate column renaming process
2025-04-23 19:22:37,491 - duckdb-integrator - INFO - Found 3 duplicate columns: ['resource_id_1', 'client_id_1', 'project_id_1']
2025-04-23 19:22:37,706 - duckdb-integrator - DEBUG - Extracted CTE names from query: ['bq_data']
2025-04-23 19:22:37,916 - duckdb-integrator - DEBUG - Found 1 JOIN patterns in query: [('bq_data', 'INNER'

,sk,client_id_bq_data,project_id_bq_data,resource_id_bq_data,project_name,CALENDAR__date,duration_hour,ts_start,ts_end,duration,id,resource_id_pg_data,client_id_pg_data,project_id_pg_data,pricing_client_id,date_from,date_to,pricing_delivery_id
0,project_id:10<>resource_id:1<>week_start:2023-...,14,10,1,Meal Planner,2023-11-27,0,2023-11-27 01:00:00+01:00,2023-12-03 01:00:00+01:00,6 days,6,1,14,10,4,2023-01-01,NaT,<NA>
1,project_id:22<>resource_id:3<>week_start:2023-...,7,22,3,[Svět Olejů] data-driven,2023-11-27,0,2023-11-27 01:00:00+01:00,2023-12-03 01:00:00+01:00,6 days,29,3,7,22,14,2023-01-01,NaT,2
2,project_id:23<>resource_id:3<>week_start:2023-...,7,23,3,[Neocity] data-driven,2023-11-27,0,2023-11-27 01:00:00+01:00,2023-12-03 01:00:00+01:00,6 days,30,3,7,23,14,2023-01-01,NaT,2
3,project_id:10<>resource_id:1<>week_start:2023-...,14,10,1,Meal Planner,2023-12-04,0,2023-12-04 01:00:00+01:00,2023-12-10 01:00:00+01:00,6 days,6,1,14,10,4,2023-01-01,NaT,<NA>
4,project_id:22<>resource_id:3<>week_start:2023-...,7,22,3,[Svět Olejů] data-driven,2023-12-04,0,2023-12-04 01:00:00+01:00,2023-12-10 01:00:00+01:00,6 days,29,3,7,22,14,2023-01-01,NaT,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
618,project_id:23<>resource_id:3<>week_start:2025-...,7,23,3,[Neocity] data-driven,2025-04-14,0,2025-04-14 02:00:00+02:00,2025-04-20 02:00:00+02:00,6 days,17,3,7,23,8,2024-02-01,NaT,<NA>
619,project_id:8<>resource_id:3<>week_start:2025-0...,10,8,3,[ABCtech.cz] data-driven,2025-04-14,0,2025-04-14 02:00:00+02:00,2025-04-20 02:00:00+02:00,6 days,9,3,10,8,8,2023-12-01,NaT,<NA>
620,project_id:22<>resource_id:3<>week_start:2025-...,7,22,3,[Svět Olejů] data-driven,2025-04-21,0,2025-04-21 02:00:00+02:00,2025-04-27 02:00:00+02:00,6 days,16,3,7,22,8,2024-02-01,NaT,<NA>
621,project_id:23<>resource_id:3<>week_start:2025-...,7,23,3,[Neocity] data-driven,2025-04-21,0,2025-04-21 02:00:00+02:00,2025-04-27 02:00:00+02:00,6 days,17,3,7,23,8,2024-02-01,NaT,<NA>


In [11]:
data

In [ ]:

logs = builder.get_performance_logs()
logs